In [8]:
from platform import python_version

print(python_version())

3.10.2


In [9]:
import pandas as pd

import pubchempy as pcp
import numpy as np
def isNaN(string):
    return string != string
import os
import glob
import re
from pybatchclassyfire import *
import csv 
import time
import json
from pandas import json_normalize
import wget
import string
import urllib.parse
import openpyxl
import statistics
import sys
from itertools import chain

In [10]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem import PandasTools

In [4]:
# write function for Spectral database results

In [5]:
input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data"
input_dir

'/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data'

In [6]:
def chemMN_CandidateSelection(df):
    
    one_df = []
    # define empty variable
    dbn= []
    for i, row in df.iterrows():
        # to compare each element with each opther element
        for j, row in df.iterrows():
            try:
                ms = [Chem.MolFromSmiles(df['SMILES'][i]), Chem.MolFromSmiles(df['SMILES'][j])]
                fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
                tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
                dbn.append({
                    'Name_i':df['ranks'][i],
                    'Name_j':df['ranks'][j],
                    'i': df['SMILES'][i],
                    'j': df['SMILES'][j],
                    'Source_i':df['Source'][i],
                    'Source_j':df['Source'][j],
                    'Tanimoto': tn
                })

            except Exception as e:
                pass
            #print(i)
            #print(j)
            #print(e)
    # save chemical similarities                    
    db_edgenode = pd.DataFrame(dbn)

    dfe = []
    heavy_atoms = ['C', 'N', 'P', 'O', 'S']
    for i, row in db_edgenode.iterrows():        
        if db_edgenode['Tanimoto'][i] >= 0.85:
            # list of mol used to calaculate the MCSS
            n = [Chem.MolFromSmiles(db_edgenode['i'][i]),Chem.MolFromSmiles(db_edgenode['j'][i])]
            res = rdFMCS.FindMCS(n)
            sm_res = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))
            # Check if the MCSS has one of the heavy atoms and whether they are
            # more than 3
            elem = [ele for ele in heavy_atoms if(ele in sm_res)]
            if elem and len(sm_res)>=3:
                MCSS_SMILES = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))

            dfe.append({
                'Start':db_edgenode['Name_i'][i],
                'End':db_edgenode['Name_j'][i],
                'Tanimoto':db_edgenode['Tanimoto'][i],
                'Start_SMILES':db_edgenode['i'][i],
                'End_SMILES':db_edgenode['j'][i],
                'Start_Source':db_edgenode['Source_i'][i],
                'End_Source':db_edgenode['Source_j'][i],
                'MCSS': MCSS_SMILES
            })
    df_edge = pd.DataFrame(dfe)
    df_edge['Start'] = df_edge['Start'].astype(str)
    df_edge['End'] = df_edge['End'].astype(str)
    df_edge['sorted_row'] = [sorted([a,b]) for a,b in zip(df_edge.Start,df_edge.End)]
    df_edge['sorted_row'] = df_edge['sorted_row'].astype(str)
    df_edge.drop_duplicates(subset=['sorted_row'], inplace=True)

    #nodes= []
    #for i, row in df.iterrows():
        #n = df['ranks'][i]
        #nodes.append({
            #'nodes':n
        #})

    #node= pd.DataFrame(nodes)

    return(df_edge)

In [7]:
def CandidateSelection_Similarity(input_dir):

    # entry is all files and folders in input_dir
    for entry in os.listdir(input_dir):
         #if the entry is also a directory
        if os.path.isdir(os.path.join(input_dir, entry)):


            # reach spectra_dereplication folder
            sub_dir_spec = input_dir + "/" + entry + '/spectral_dereplication/'
            # reach SIRIUS results
            sub_dir_sir = input_dir + "/" + entry + '/insilico/SIRIUS/'

            #list of all csv files in the spectral dereplication foler
            spec_msp_csv = (glob.glob(input_dir + "/" + entry + '/spectral_dereplication' +'/*.csv'))
            # Sirius csv result file
            sir_msp_csv = input_dir + "/" + entry + "/insilico/MS1DATA.csv"

            # if both exist; which should be the case, even in case of 0 results
            if os.path.exists(sir_msp_csv) and os.path.exists(spec_msp_csv[0]):

                # read both csv files
                spec_msv = pd.read_csv(spec_msp_csv[0])
                sir_msv = pd.read_csv(sir_msp_csv)



                spec_msv = spec_msv[[
                    'id_X',
                    'premz',
                    'rtmin',
                    'rtmax',
                    'rtmed',
                    'rtmean',
                    'col_eng',
                    'pol',
                    'int',
                    'source_file',
                    'mbank_results_csv',
                    ]]
                sir_msv = sir_msv[['id_X',
                    'premz',
                    'rtmed',
                    'rtmean',
                    'int',
                    'col_eng',
                    'pol',
                    'ms2Peaks',
                    'ms1Peaks',
                    'sirius_result_dir']]

                merged_df = sir_msv.merge(spec_msv, 
                              how='inner', 
                              left_on=['premz', 'rtmed','rtmean','int','col_eng','pol'], 
                              right_on=['premz','rtmed','rtmean','int','col_eng','pol'])

                for mer, rows in merged_df.iterrows():

                    sirius_csv = merged_df["sirius_result_dir"][mer].replace("./", input_dir+"/")
                    mbank_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/")
                    gnps_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/").replace('mbank', 'gnps').replace('MassBank', 'GNPS')
                    hmdb_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/").replace('mbank', 'hmdb').replace('MassBank', 'HMDB')


                    if os.path.exists(sirius_csv) and os.path.exists(gnps_csv) and os.path.exists(mbank_csv) and os.path.exists(hmdb_csv):


                        sirius_df = pd.read_csv(sirius_csv)
                        sirius_df = sirius_df.drop_duplicates('smiles')

                        mbank_df = pd.read_csv(mbank_csv)
                        mbank_df = mbank_df.drop_duplicates('MBSMILES')

                        gnps_df = pd.read_csv(gnps_csv)
                        gnps_df = gnps_df.drop_duplicates('GNPSSMILES')

                        hmdb_df = pd.read_csv(hmdb_csv)
                        hmdb_df = hmdb_df.drop_duplicates('HMDBSMILES')

                        #1 SGHM
                        if len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                            mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(sirius_df["Source"])), 
                               *(list(gnps_df["Source"]))
                               ,*(list(mbank_df["Source"])),
                                        *(list(hmdb_df["Source"]))]

                            rank_l2 = [*(list(sirius_df["rank_ids"])), 
                               *(list(gnps_df["rank_ids"]))
                               ,*(list(mbank_df["rank_ids"])), 
                                      *(list(hmdb_df["rank_ids"]))]

                            smiles_l3 = [*(list(sirius_df["smiles"])), 
                               *(list(gnps_df["GNPSSMILES"]))
                               ,*(list(mbank_df["MBSMILES"])),
                                        *(list(hmdb_df["HMDBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])
                            
                            
                            
                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            one_candidate = one_candidate_selection(sm, Source = "SGHM")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')


                        #2 SGM
                        elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:

                            mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(sirius_df["Source"])), 
                               *(list(gnps_df["Source"]))
                               ,*(list(mbank_df["Source"]))]

                            rank_l2 = [*(list(sirius_df["rank_ids"])), 
                               *(list(gnps_df["rank_ids"]))
                               ,*(list(mbank_df["rank_ids"]))]

                            smiles_l3 = [*(list(sirius_df["smiles"])), 
                               *(list(gnps_df["GNPSSMILES"]))
                               ,*(list(mbank_df["MBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            one_candidate = one_candidate_selection(sm, Source = "SGM")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                        #3 SHM
                        elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:

                            mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(sirius_df["Source"]))
                               ,*(list(mbank_df["Source"])),
                                        *(list(hmdb_df["Source"]))]

                            rank_l2 = [*(list(sirius_df["rank_ids"]))
                               ,*(list(mbank_df["rank_ids"])), 
                                      *(list(hmdb_df["rank_ids"]))]

                            smiles_l3 = [*(list(sirius_df["smiles"]))
                               ,*(list(mbank_df["MBSMILES"])),
                                        *(list(hmdb_df["HMDBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            one_candidate = one_candidate_selection(sm, Source = "SHM")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                        #4 SGH
                        elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                            #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(sirius_df["Source"]))
                               ,*(list(gnps_df["Source"])),
                                        *(list(hmdb_df["Source"]))]

                            rank_l2 = [*(list(sirius_df["rank_ids"]))
                               ,*(list(gnps_df["rank_ids"])), 
                                      *(list(hmdb_df["rank_ids"]))]

                            smiles_l3 = [*(list(sirius_df["smiles"]))
                               ,*(list(gnps_df["GNPSSMILES"])),
                                        *(list(hmdb_df["HMDBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])

                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            one_candidate = one_candidate_selection(sm, Source = "SGH")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                            
                        #5 GHM
                        elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                            mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            #sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(gnps_df["Source"]))
                               ,*(list(mbank_df["Source"])),
                                        *(list(hmdb_df["Source"]))]

                            rank_l2 = [*(list(gnps_df["rank_ids"]))
                               ,*(list(mbank_df["rank_ids"])), 
                                      *(list(hmdb_df["rank_ids"]))]

                            smiles_l3 = [*(list(gnps_df["GNPSSMILES"]))
                               ,*(list(mbank_df["MBSMILES"])),
                                        *(list(hmdb_df["HMDBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])

                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            one_candidate = one_candidate_selection(sm, Source = "GHM")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                            
                        #6 SG
                        elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:
                            #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(sirius_df["Source"])), 
                               *(list(gnps_df["Source"]))]
                               #,*(list(mbank_df["Source"]))]

                            rank_l2 = [*(list(sirius_df["rank_ids"])), 
                               *(list(gnps_df["rank_ids"]))]
                               #,*(list(mbank_df["rank_ids"]))]

                            smiles_l3 = [*(list(sirius_df["smiles"])), 
                               *(list(gnps_df["GNPSSMILES"]))]
                               #,*(list(mbank_df["MBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            
                            one_candidate = one_candidate_selection(sm, Source = "SG")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                        #7 SH
                        elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                            #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(sirius_df["Source"])),
                                        *(list(hmdb_df["Source"]))]

                            rank_l2 = [*(list(sirius_df["rank_ids"])), 
                                      *(list(hmdb_df["rank_ids"]))]

                            smiles_l3 = [*(list(sirius_df["smiles"])),
                                        *(list(hmdb_df["HMDBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])
                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            
                            one_candidate = one_candidate_selection(sm, Source = "SH")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                        #8 SM
                        elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:
                            mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            #hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(sirius_df["Source"]))
                               ,*(list(mbank_df["Source"]))]

                            rank_l2 = [*(list(sirius_df["rank_ids"]))
                               ,*(list(mbank_df["rank_ids"]))]

                            smiles_l3 = [*(list(sirius_df["smiles"]))
                               ,*(list(mbank_df["MBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            
                            one_candidate = one_candidate_selection(sm, Source = "SM")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                        #9 GM
                        elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:
                            mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            #hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            #sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(mbank_df["Source"])),
                                        *(list(gnps_df["Source"]))]

                            rank_l2 = [*(list(mbank_df["rank_ids"])), 
                                      *(list(gnps_df["rank_ids"]))]

                            smiles_l3 = [*(list(mbank_df["MBSMILES"])),
                                        *(list(gnps_df["GNPSSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            
                            one_candidate = one_candidate_selection(sm, Source = "GM")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                        #10 GH
                        elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                            #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            #sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(gnps_df["Source"])),
                                        *(list(hmdb_df["Source"]))]

                            rank_l2 = [*(list(gnps_df["rank_ids"])),
                                      *(list(hmdb_df["rank_ids"]))]

                            smiles_l3 = [*(list(gnps_df["GNPSSMILES"])),
                                        *(list(hmdb_df["HMDBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])
                            df_edge = chemMN_CandidateSelection(sm)



                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            one_candidate = one_candidate_selection(sm, Source = "GH")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                        #11 HM
                        elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                            mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                            #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                            hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                            #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                            #sirius_df["Source"] = "SIRIUS"


                            source_l1 = [*(list(mbank_df["Source"])),
                                        *(list(hmdb_df["Source"]))]

                            rank_l2 = [*(list(mbank_df["rank_ids"])), 
                                      *(list(hmdb_df["rank_ids"]))]

                            smiles_l3 = [*(list(mbank_df["MBSMILES"])),
                                        *(list(hmdb_df["HMDBSMILES"]))]

                            sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                            df_edge = chemMN_CandidateSelection(sm)

                            df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                            one_candidate = one_candidate_selection(sm, Source = "HM")
                            one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                        # S
                        #elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:

                        # G
                        #elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:

                        # M
                        #elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:

                        # H
                        #elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:

In [15]:

# entry is all files and folders in input_dir
for entry in os.listdir(input_dir):
     #if the entry is also a directory
    if os.path.isdir(os.path.join(input_dir, entry)):


        # reach spectra_dereplication folder
        sub_dir_spec = input_dir + "/" + entry + '/spectral_dereplication/'
        # reach SIRIUS results
        sub_dir_sir = input_dir + "/" + entry + '/insilico/SIRIUS/'

        #list of all csv files in the spectral dereplication foler
        spec_msp_csv = (glob.glob(input_dir + "/" + entry + '/spectral_dereplication' +'/*.csv'))
        # Sirius csv result file
        sir_msp_csv = input_dir + "/" + entry + "/insilico/MS1DATA.csv"

        # if both exist; which should be the case, even in case of 0 results
        if os.path.exists(sir_msp_csv) and os.path.exists(spec_msp_csv[0]):

            # read both csv files
            spec_msv = pd.read_csv(spec_msp_csv[0])
            sir_msv = pd.read_csv(sir_msp_csv)



            spec_msv = spec_msv[[
                'id_X',
                'premz',
                'rtmin',
                'rtmax',
                'rtmed',
                'rtmean',
                'col_eng',
                'pol',
                'int',
                'source_file',
                'mbank_results_csv',
                ]]
            sir_msv = sir_msv[['id_X',
                'premz',
                'rtmed',
                'rtmean',
                'int',
                'col_eng',
                'pol',
                'ms2Peaks',
                'ms1Peaks',
                'sirius_result_dir']]

            merged_df = sir_msv.merge(spec_msv, 
                          how='inner', 
                          left_on=['premz', 'rtmed','rtmean','int','col_eng','pol'], 
                          right_on=['premz','rtmed','rtmean','int','col_eng','pol'])

            for mer, rows in merged_df.iterrows():

                sirius_csv = merged_df["sirius_result_dir"][mer].replace("./", input_dir+"/")
                mbank_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/")
                gnps_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/").replace('mbank', 'gnps').replace('MassBank', 'GNPS')
                hmdb_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/").replace('mbank', 'hmdb').replace('MassBank', 'HMDB')


                if os.path.exists(sirius_csv) and os.path.exists(gnps_csv) and os.path.exists(mbank_csv) and os.path.exists(hmdb_csv):


                    sirius_df = pd.read_csv(sirius_csv)
                    sirius_df = sirius_df.drop_duplicates('smiles')

                    mbank_df = pd.read_csv(mbank_csv)
                    mbank_df = mbank_df.drop_duplicates('MBSMILES')

                    gnps_df = pd.read_csv(gnps_csv)
                    gnps_df = gnps_df.drop_duplicates('GNPSSMILES')

                    hmdb_df = pd.read_csv(hmdb_csv)
                    hmdb_df = hmdb_df.drop_duplicates('HMDBSMILES')

                    #1 SGHM
                    if len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"])), 
                           *(list(gnps_df["Source"]))
                           ,*(list(mbank_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"])), 
                           *(list(gnps_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"])), 
                           *(list(gnps_df["GNPSSMILES"]))
                           ,*(list(mbank_df["MBSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])



                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                        one_candidate = one_candidate_selection(sm, Source = "SGHM")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')


                    #2 SGM
                    elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:

                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"])), 
                           *(list(gnps_df["Source"]))
                           ,*(list(mbank_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"])), 
                           *(list(gnps_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"])), 
                           *(list(gnps_df["GNPSSMILES"]))
                           ,*(list(mbank_df["MBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                        one_candidate = one_candidate_selection(sm, Source = "SGM")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                    #3 SHM
                    elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:

                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"]))
                           ,*(list(mbank_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"]))
                           ,*(list(mbank_df["MBSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                        one_candidate = one_candidate_selection(sm, Source = "SHM")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                    #4 SGH
                    elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                        #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"]))
                           ,*(list(gnps_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"]))
                           ,*(list(gnps_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"]))
                           ,*(list(gnps_df["GNPSSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])

                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                        one_candidate = one_candidate_selection(sm, Source = "SGH")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                    #5 GHM
                    elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        #sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(gnps_df["Source"]))
                           ,*(list(mbank_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(gnps_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(gnps_df["GNPSSMILES"]))
                           ,*(list(mbank_df["MBSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])

                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                        one_candidate = one_candidate_selection(sm, Source = "GHM")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                    #6 SG
                    elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:
                        #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"])), 
                           *(list(gnps_df["Source"]))]
                           #,*(list(mbank_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"])), 
                           *(list(gnps_df["rank_ids"]))]
                           #,*(list(mbank_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"])), 
                           *(list(gnps_df["GNPSSMILES"]))]
                           #,*(list(mbank_df["MBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')

                        one_candidate = one_candidate_selection(sm, Source = "SG")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                    #7 SH
                    elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                        #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])
                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')

                        one_candidate = one_candidate_selection(sm, Source = "SH")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                    #8 SM
                    elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        #hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"]))
                           ,*(list(mbank_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"]))
                           ,*(list(mbank_df["MBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')

                        one_candidate = one_candidate_selection(sm, Source = "SM")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                    #9 GM
                    elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        #hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        #sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(mbank_df["Source"])),
                                    *(list(gnps_df["Source"]))]

                        rank_l2 = [*(list(mbank_df["rank_ids"])), 
                                  *(list(gnps_df["rank_ids"]))]

                        smiles_l3 = [*(list(mbank_df["MBSMILES"])),
                                    *(list(gnps_df["GNPSSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')

                        one_candidate = one_candidate_selection(sm, Source = "GM")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                    #10 GH
                    elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                        #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        #sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(gnps_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(gnps_df["rank_ids"])),
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(gnps_df["GNPSSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])
                        df_edge = chemMN_CandidateSelection(sm)



                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                        one_candidate = one_candidate_selection(sm, Source = "GH")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')
                    #11 HM
                    elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        #sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(mbank_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(mbank_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(mbank_df["MBSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_ChemMNedges.tsv", sep='\t')
                        one_candidate = one_candidate_selection(sm, Source = "HM")
                        print(one_candidate)
                        one_candidate.to_csv(input_dir + "/" + entry + "/" + str(merged_df["premz"][mer]) + "_one_candidate_list.tsv", sep='\t')

                    # S
                    #elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:

                    # G
                    #elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:

                    # M
                    #elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:

                    # H
                    #elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:

      Source ranks                 SMILES SIRIUS GNPS MassBank  HMDB  Count
0     SIRIUS   S_1    C(C(CS(=O)(=O)O)N)O    S_1  NaN      NaN   NaN    1.0
1     SIRIUS   S_2    C(C(CS(=O)(=O)O)O)N    S_2  NaN      NaN   NaN    1.0
2     SIRIUS   S_3    C(C(CO)S(=O)(=O)O)N    S_3  NaN      NaN   NaN    1.0
3     SIRIUS   S_4    C(CS(=O)(=O)O)C(N)O    S_4  NaN      NaN   NaN    1.0
4     SIRIUS   S_5      C(CNO)CS(=O)(=O)O    S_5  NaN      NaN   NaN    1.0
5     SIRIUS   S_6    C(CO)C(N)S(=O)(=O)O    S_6  NaN      NaN   NaN    1.0
6     SIRIUS   S_7      C(CO)NCS(=O)(=O)O    S_7  NaN      NaN   NaN    1.0
7     SIRIUS   S_8      C(CS(=O)(=O)O)NCO    S_8  NaN      NaN   NaN    1.0
8     SIRIUS   S_9    C(CN)C(O)S(=O)(=O)O    S_9  NaN      NaN   NaN    1.0
9     SIRIUS  S_10      C(CN)COS(=O)(=O)O   S_10  NaN      NaN   NaN    1.0
10    SIRIUS  S_11      C(CON)CS(=O)(=O)O   S_11  NaN      NaN   NaN    1.0
11    SIRIUS  S_12      C(CNS(=O)(=O)O)CO   S_12  NaN      NaN   NaN    1.0
12    SIRIUS

      Source ranks                                             SMILES SIRIUS  \
0     SIRIUS   S_1                                   C1CCNC(C1)C(=O)O    S_1   
1     SIRIUS   S_2                                [CH2-]N1CCCC1C(=O)O    S_2   
2     SIRIUS   S_3                                     CN1CCCC1C(=O)O    S_3   
3     SIRIUS   S_4                                   CCC1CC(N1)C(=O)O    S_4   
4     SIRIUS   S_5                                   CC1CCC(N1)C(=O)O    S_5   
5     SIRIUS   S_6                                [CH2+]CCCC(C(=O)O)N    S_6   
6     SIRIUS   S_7                                     CC1CCNC1C(=O)O    S_7   
7     SIRIUS   S_8                                   C1CCN(C1)CC(=O)O    S_8   
8     SIRIUS   S_9                                   C1CC(C1)NCC(=O)O    S_9   
9     SIRIUS  S_10                                   CCCC1C(N1)C(=O)O   S_10   
10    SIRIUS  S_11                                   C1CC(NC1)CC(=O)O   S_11   
11    SIRIUS  S_12                      

   Source ranks                              SMILES SIRIUS GNPS  MassBank  \
0  SIRIUS   S_1      CCC(=O)OC(CC(=O)O)C[N+](C)(C)C    S_1  G_1       NaN   
1  SIRIUS   S_2      CC(=O)OC(CC(=O)OC)C[N+](C)(C)C    S_2  NaN       NaN   
2  SIRIUS   S_3           CCCC(=O)OC(CC(=O)O)CN(C)C    S_3  NaN       NaN   
3  SIRIUS   S_4      CC(=O)COC(CC(=O)O)C[N+](C)(C)C    S_4  NaN       NaN   
4  SIRIUS   S_5      CC[N+](C)(C)CC(CC(=O)O)OC(=O)C    S_5  NaN       NaN   
5  SIRIUS   S_6    CC(=O)OC(C)(CC(=O)O)C[N+](C)(C)C    S_6  NaN       NaN   
6    GNPS   G_1  CCC(=O)O[C@H](CC(O)=O)C[N+](C)(C)C    S_1  G_1       NaN   
7    GNPS   G_2   CCC(=O)OC(CC(=O)[O-])C[N+](C)(C)C    NaN  G_2       NaN   

   HMDB  Count  
0   NaN    2.0  
1   NaN    1.0  
2   NaN    1.0  
3   NaN    1.0  
4   NaN    1.0  
5   NaN    1.0  
6   NaN    2.0  
7   NaN    1.0  
      Source ranks                                             SMILES SIRIUS  \
0     SIRIUS   S_1                                   C1CCNC(C1)C(=O)O    

      Source ranks                               SMILES SIRIUS GNPS MassBank  \
0     SIRIUS   S_1             CN(C)C(=NCCCC(C(=O)O)N)N    S_1  G_1      NaN   
1     SIRIUS   S_2               CCNC(=NCCCC(C(=O)O)N)N    S_2  NaN      NaN   
2     SIRIUS   S_3               CNC(=NC)NCCCC(C(=O)O)N    S_3  NaN      NaN   
3     SIRIUS   S_4             CN=C(N)N(C)CCCC(C(=O)O)N    S_4  NaN      NaN   
4     SIRIUS   S_5          C[N+](=C(N)NCCCC(C(=O)O)N)C    S_5  NaN      NaN   
5     SIRIUS   S_6               CN=C(N)NCCCC(C(=O)OC)N    S_6  NaN      NaN   
6     SIRIUS   S_7               CN=C(N)NCCCCC(C(=O)O)N    S_7  NaN      NaN   
7     SIRIUS   S_8               CCOC(=O)C(CCCN=C(N)N)N    S_8  NaN      NaN   
8     SIRIUS   S_9               CN(C)NC=NCCCC(C(=O)O)N    S_9  NaN      NaN   
9     SIRIUS  S_10             CN(C)C(CCCN=C(N)N)C(=O)O   S_10  NaN      NaN   
10    SIRIUS  S_11             CN(C)C(=NC(CCCN)C(=O)O)N   S_11  NaN      NaN   
11    SIRIUS  S_12             C(CCC(C(=

      Source ranks                                             SMILES SIRIUS  \
0     SIRIUS   S_1                                C(C(CS(=O)(=O)O)N)O    S_1   
1     SIRIUS   S_2                                C(C(CS(=O)(=O)O)O)N    S_2   
2     SIRIUS   S_3                                C(CS(=O)(=O)O)C(N)O    S_3   
3     SIRIUS   S_4                                  C(CNO)CS(=O)(=O)O    S_4   
4     SIRIUS   S_5                                C(CO)C(N)S(=O)(=O)O    S_5   
5     SIRIUS   S_6                                C(C(CO)S(=O)(=O)O)N    S_6   
6     SIRIUS   S_7                                C(CN)C(O)S(=O)(=O)O    S_7   
7     SIRIUS   S_8                                  C(CO)NCS(=O)(=O)O    S_8   
8     SIRIUS   S_9                                  C(CS(=O)(=O)O)NCO    S_9   
9     SIRIUS  S_10                                  C(CN)COS(=O)(=O)O   S_10   
10    SIRIUS  S_11                                  C(CNS(=O)(=O)O)CO   S_11   
11    SIRIUS  S_12                      

In [14]:
one_candidate_selection(sm, Source = "SGM")

,Source,ranks,SMILES,SIRIUS,GNPS,MassBank,HMDB,Count
0,SIRIUS,S_1,C(C(CS(=O)(=O)O)N)O,S_1,NaN,NaN,NaN,1.0
1,SIRIUS,S_2,C(C(CS(=O)(=O)O)O)N,S_2,NaN,NaN,NaN,1.0
2,SIRIUS,S_3,C(CS(=O)(=O)O)C(N)O,S_3,NaN,NaN,NaN,1.0
3,SIRIUS,S_4,C(CNO)CS(=O)(=O)O,S_4,NaN,NaN,NaN,1.0
4,SIRIUS,S_5,C(CO)C(N)S(=O)(=O)O,S_5,NaN,NaN,NaN,1.0
5,SIRIUS,S_6,C(C(CO)S(=O)(=O)O)N,S_6,NaN,NaN,NaN,1.0
6,SIRIUS,S_7,C(CN)C(O)S(=O)(=O)O,S_7,NaN,NaN,NaN,1.0
7,SIRIUS,S_8,C(CO)NCS(=O)(=O)O,S_8,NaN,NaN,NaN,1.0
8,SIRIUS,S_9,C(CS(=O)(=O)O)NCO,S_9,NaN,NaN,NaN,1.0
9,SIRIUS,S_10,C(CN)COS(=O)(=O)O,S_10,NaN,NaN,NaN,1.0


In [12]:
def one_candidate_selection(df, Source = "SGHM"):
    df["SIRIUS"] = np.nan
    df["GNPS"] = np.nan
    df["MassBank"] = np.nan
    df["HMDB"] = np.nan
    df["Count"] = np.nan

    for smiles, rows in df.iterrows():

        if Source == "SGHM" or Source == "SGM" or Source == "SGH" or Source == "SHM" or Source == "SG" or Source == "SM" or Source == "SH":
            for sirius_i, row in sirius_df.iterrows():
                try:
                    ms = [Chem.MolFromSmiles(df["SMILES"][smiles]), Chem.MolFromSmiles(sirius_df["smiles"][sirius_i])]
                    fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
                    tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
                    if tn >= 0.99:
                        if isNaN(df["SIRIUS"][smiles]):

                            df.loc[smiles, "SIRIUS"] = sirius_df["rank_ids"][sirius_i]

                        else:
                            df.loc[smiles, "SIRIUS"] = str(df["SIRIUS"][smiles]) + ", "+ sirius_df["rank_ids"][sirius_i]
                            
                        if isNaN(df["Count"][smiles]):
                            df.loc[smiles, "Count"] = 1
                        else:
                            df.loc[smiles, "Count"] = df["Count"][smiles] + 1
                except:
                    pass

        if Source == "SGHM" or Source == "SGM" or Source == "SGH" or Source == "GHM" or Source == "SG" or Source == "GM" or Source == "GH":


            for gnps_i, row in gnps_df.iterrows():
                try:
                    ms = [Chem.MolFromSmiles(df["SMILES"][smiles]), Chem.MolFromSmiles(gnps_df["GNPSSMILES"][gnps_i])]
                    fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
                    tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])

                    if tn >= 0.99:
                        if isNaN(df["GNPS"][smiles]):

                            df.loc[smiles, "GNPS"] = gnps_df["rank_ids"][gnps_i]

                        else:
                            df.loc[smiles, "GNPS"] = str(df["GNPS"][smiles]) + ", "+ gnps_df["rank_ids"][gnps_i]
                            
                            
                        if isNaN(df["Count"][smiles]):
                            df.loc[smiles, "Count"] = 1
                        else:
                            df.loc[smiles, "Count"] = df["Count"][smiles] + 1
                except:
                    pass


        if Source == "SGHM" or Source == "SGM" or Source == "SHM" or Source == "GHM" or Source == "SM" or Source == "GM" or Source == "HM":
            for mbank_i, row in mbank_df.iterrows():
                try:
                    ms = [Chem.MolFromSmiles(df["SMILES"][smiles]), Chem.MolFromSmiles(mbank_df["MBSMILES"][mbank_i])]
                    fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
                    tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
                    if tn >= 0.99:
                        if isNaN(df["MassBank"][smiles]):

                            df.loc[smiles, "MassBank"] = mbank_df["rank_ids"][mbank_i]

                        else:
                            df.loc[smiles, "MassBank"] = str(df["MassBank"][smiles]) + ", "+ mbank_df["rank_ids"][mbank_i]
                            
                        if isNaN(df["Count"][smiles]):
                            df.loc[smiles, "Count"] = 1
                        else:
                            df.loc[smiles, "Count"] = df["Count"][smiles] + 1
                except:
                    pass



        if Source == "SGHM" or Source == "SGH" or Source == "SHM" or Source == "GHM" or Source == "SH" or Source == "GH" or Source == "HM":


            for hmdb_i, row in hmdb_df.iterrows():
                try:
                    ms = [Chem.MolFromSmiles(df["SMILES"][smiles]), Chem.MolFromSmiles(hmdb_df["HMDBSMILES"][hmdb_i])]
                    fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
                    tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
                    if tn >= 0.99:
                        if isNaN(df["HMDB"][smiles]):

                            df.loc[smiles, "HMDB"] = hmdb_df["rank_ids"][hmdb_i]

                        else:
                            df.loc[smiles, "HMDB"] = str(df["HMDB"][smiles]) + ", "+ hmdb_df["rank_ids"][hmdb_i]
                            
                        if isNaN(df["Count"][smiles]):
                            df.loc[smiles, "Count"] = 1
                        else:
                            df.loc[smiles, "Count"] = df["Count"][smiles] + 1
                except:
                    pass
 
    return(df)

In [ ]:
#1 SGHM
#2 SGM
#3 SHM
#4 SGH
#5 GHM
#6 SG
#7 SH
#8 SM
#9 GM
#10 GH
#11 HM

In [6]:
one_can = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data/VN_211016_acetyl_carnitine/204.122756958008_one_candidate_list.tsv", sep = "\t")

In [8]:
del one_can["Count"]

In [10]:
df =one_can

In [25]:
df[df.eq(df["SIRIUS"], axis=0)]

,Unnamed: 0,Source,ranks,SMILES,SIRIUS,GNPS,MassBank,HMDB
0,NaN,NaN,S_1,NaN,S_1,NaN,NaN,NaN
1,NaN,NaN,S_2,NaN,S_2,NaN,NaN,NaN
2,NaN,NaN,S_3,NaN,S_3,NaN,NaN,NaN
3,NaN,NaN,S_4,NaN,S_4,NaN,NaN,NaN
4,NaN,NaN,S_5,NaN,S_5,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,S_1,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
